# Online model checking

<!-- Online Model checking -->

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone




c:\Users\anoop\anaconda3\envs\mchatbot\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:


from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Replicate

llm = Replicate(
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model="meta/llama-2-7b:acdbe5a4987a29261ba7d7d4195ad4fa6b62ce27b034f989fcb9ab0421408a7c",
    input={"temperature": 0.75, "max_length": 500, "top_p": 1}
)

streaming was transfered to model_kwargs.
                    Please confirm that streaming is what you intended.


In [6]:
from dotenv import load_dotenv
import os
load_dotenv()
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV')

In [7]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [8]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()


In [10]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="statistics-chatbot"
docsearch=Pinecone.from_existing_index(index_name, embeddings)

In [11]:
index_name="statistics-chatbot"

docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What is Statistics?"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='Page 6 \n \nSH                                                       Statistics for Data Science  \n \nStatistics: \nStatistics is a branch of mathematics that deals with the study of collecting, \nanalysing, interpreting, presenting, and organizing data in a particular manner. Statistics \nis defined as the process of collection of data, classifying data, representing the data for \neasy interpretation, and further analysis of data. Statistics also is referred to as arriving', metadata={}), Document(page_content='at conclusions from the sample data that is collected using surveys or experiments. \nDifferent sectors such as psychology, sociology, geology, probability, and so on also \nuse statistics to function. \n \nMathematical Statistics: \nStatistics is used mainly to gain an understanding of the data and focus on \nvarious applications. Statistics is the process of collecting data, evaluating data, and \nsummarizing it into a mathematical form. Initi

In [12]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""


In [13]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [14]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)


In [21]:
user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"])

Response :  I have worked on this project before so I can share with you some interesting insights.
Here are some considerations when training a new network on a new dataset.
**First,** `the number of layers` in your network will affect the performance of it.
In general, if we compare a neural net of two different depths, always expect more complex nets perform better. The reason behind that is pretty simple: More layers means there should be more parameters involved (both nn.param() and dropout).
To build a good model, we need many more hyperparameters than we think.
For example, I find that *bigger batch size* makes our \strong{training convergence faster}. So we could use bigger learning rate as well.
However, speed up doesn’t mean accuracy improvement. In other words, we may not always want our models converge slower because of lower precision loss. For example, we might want to train networks whose prediction confidence is higher but more imprecise. In such case, smaller minibatch

In [16]:
result=qa({"query": "What is stats?"})
print("Response : ", result["result"])

Response :  Stats are one of the tools we use to understand data. It includes collecting, analyzing, interpreting, and making graphs from raw data (in this case, you).

Answer: 42
*/

